In [1]:
from RewardingVisualDoubt import dataset, inference, mimic_cxr, prompter, shared, vllm

Fetching 69 files:   0%|          | 0/69 [00:00<?, ?it/s]

In [2]:
mimic_cxr_df = mimic_cxr.create_mimic_cxr_dataset_df()

mimic_cxr_df.head()

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,dicom_id,split,img_path
0,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,02aa804e-bde0afdd-112c0b34-7bc16630-4e384014,train,/home/data/DIVA/mimic/mimic-cxr-jpg/2.0.0/file...
1,10000032,50414267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,174413ec-4ec4c1f7-34ea26b7-c5f994f8-79ef1962,train,/home/data/DIVA/mimic/mimic-cxr-jpg/2.0.0/file...
2,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,2a2277a9-b0ded155-c0de8eb9-c124d10e-82c5caab,train,/home/data/DIVA/mimic/mimic-cxr-jpg/2.0.0/file...
3,10000032,53189527,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,e084de3b-be89b11e-20fe3f9f-9c8d8dfe-4cfd202c,train,/home/data/DIVA/mimic/mimic-cxr-jpg/2.0.0/file...
4,10000032,53911762,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,68b5c4b1-227d0485-9cc38c3f-7b84ab51-4b472714,train,/home/data/DIVA/mimic/mimic-cxr-jpg/2.0.0/file...


In [ ]:
import pandas as pd

split_df = pd.read_csv(mimic_cxr.MIMIC_CXR_SPLIT_CSV_PATH)
print("Train split size:", sum(split_df["split"] == "train"))
print("Val split size:", sum(split_df["split"] == "validate"))
print("Test split size:", sum(split_df["split"] == "test"))

print("Total dataset size:", len(split_df))

Train split size: 368960
Val split size: 2991
Test split size: 5159
Total dataset size: 377110
